### TODO

- [x] Initial code
- [ ] Add theory, formulas
- [ ] Explain how it works
- [ ] Compare to classic ttest 
- [ ] Run Monte-Carlo, check criteria

In [3]:
import numpy as np
import scipy.stats as sps

In [44]:
def relative_ttest(control, test, alpha=0.05):
    """ Student's T test for relative difference with confidence interval for two sided hypotesis"""
    
    # Sample means
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    
    # SEM = Standard Error of the Mean aka. std for means for given sample size
    sem_control  = np.var(control) / len(control) 
    sem_test  = np.var(test) / len(test)
    
    # Differences
    difference_mean = mean_test - mean_control
    difference_sem  = sem_test + sem_control
    relative_mu = difference_mean / mean_control
    
    covariance = -sem_control
    
    # Taylors linearization to get corrent variance estimation for relative difference
    relative_var =  difference_sem / (mean_control ** 2) \
                    + ((difference_mean ** 2)  / (mean_control ** 4)) * sem_control \
                    - 2 * (difference_mean / (mean_control ** 3)) * covariance
    
    relative_distribution = sps.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([alpha / 2, 1 - alpha/2])
    
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    return {"p-value": pvalue, 
            "effect": relative_mu, 
            "ci_length": ci_length, 
            "left_bound": left_bound,
            "right_bound": right_bound}

In [45]:
control = sps.expon().rvs(1000)
test = sps.expon().rvs(1000)

relative_ttest(control, test)

{'p-value': 0.16423043316840946,
 'effect': -0.05968870331736494,
 'ci_length': 0.1682086230232332,
 'left_bound': -0.14379301482898155,
 'right_bound': 0.02441560819425164}